In [4]:
import pandas as pd
from bs4 import BeautifulSoup


from article_collection import ArticleCollector, ArticleScraper

In [26]:
for n in range(len(stories)):
    try:
        stories[n].div.div.find_all('span',{'data-testid':'timestamp'})[0].text
    except:
        print(stories[n].find_all('span',{'data-testid':'timestamp'}))

[<span class="wpds-c-jKQbzV font-xxxs font-light font--meta-text lh-sm gray-dark dot-xxs-gray-dark" data-testid="timestamp">November 20, 2024</span>]
[<span class="wpds-c-jKQbzV font-xxxs font-light font--meta-text lh-sm gray-dark dot-xxs-gray-dark" data-testid="timestamp">November 19, 2024</span>]
[<span class="wpds-c-jKQbzV font-xxxs font-light font--meta-text lh-sm gray-dark dot-xxs-gray-dark" data-testid="timestamp">November 18, 2024</span>]
[<span class="wpds-c-jKQbzV font-xxxs font-light font--meta-text lh-sm gray-dark dot-xxs-gray-dark" data-testid="timestamp">November 18, 2024</span>]


In [ ]:
with open('data/wpost.html', 'r') as fpath:
    html = fpath.read()

soup = BeautifulSoup(html, 'html.parser')

stories = soup.find_all('div', {'data-feature-id':'homepage/story'})

urls = [stories[n].div.div.a['href'] + '.com' for n in range(len(stories))]
dates = [stories[n].find_all('span',{'data-testid':'timestamp'})[0].text for n in range(len(stories))]
descriptions = [None for n in range(len(stories))]
sources = ['Washington Post' for n in range(len(stories))]
titles = [stories[n].find_all('h3', {'data-qa':'card-title'})[0].text for n in range(len(stories))]


In [32]:
len(titles)

584

In [2]:
collector = ArticleCollector(API_KEY='b0ad6680b01344f38334d267c9bf1da2')

collector.scrape_wpost()

0
1


ElementClickInterceptedException: Message: element click intercepted: Element <iframe id="google_ads_iframe_/701/wpni.politics/front_0" name="google_ads_iframe_/701/wpni.politics/front_0" title="3rd party ad content" width="0" height="250" scrolling="no" marginwidth="0" marginheight="0" frameborder="0" aria-label="Advertisement" tabindex="0" allow="private-state-token-redemption;attribution-reporting" sandbox="allow-forms allow-pointer-lock allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts" data-load-complete="true" data-google-container-id="1" style="border: 0px; vertical-align: bottom; min-width: 100%;" data-integralas-id-4865ac4c-1a14-ee96-7192-a9c59fbdc809=""></iframe> is not clickable at point (600, 200). Other element would receive the click: <h1 class="font-lg mt-sm pt-xs pb-xs center font--headline offblack" id="HEADER" style="-webkit-font-smoothing: antialiased; word-break: break-word;">...</h1>
  (Session info: chrome=131.0.6778.86)
Stacktrace:
0   c